In [1]:
import os
import sys
sys.path.append("/home/bydguikong/yy_ws/nuplan-devkit")
devkit_path = "/home/bydguikong/yy_ws/nuplan-devkit"

# 添加到 Python 模块搜索路径中（对 import 起效）
if devkit_path not in sys.path:
    sys.path.append(devkit_path)
!pip list  | grep nuplan

In [2]:
import os
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder import NuPlanScenarioBuilder
from nuplan.planning.scenario_builder.scenario_filter import ScenarioFilter
from nuplan.planning.utils.multithreading.worker_parallel import SingleMachineParallelExecutor

# --- Hardcoded Correct Paths ---
db_files_path = '/home/bydguikong/nuplan/dataset/nuplan-v1.1/splits/mini'
map_path = '/home/bydguikong/nuplan/dataset/maps'
map_version = "nuplan-maps-v1.0"

print("--- Minimal Test Script (v2) ---")
print(f"Attempting to load data from: {db_files_path}")
print(f"Attempting to load maps from: {map_path}")

try:
    # 1. Initialize the builder with the missing sensor_root argument
    builder = NuPlanScenarioBuilder(
        data_root=db_files_path,
        map_root=map_path,
        sensor_root=None, # <-- 这是修正后增加的一行
        db_files=None,
        map_version=map_version
    )

    # 2. Create a filter that accepts everything
    scenario_filter = ScenarioFilter(
        scenario_types=None,
        scenario_tokens=None,
        log_names=None,
        map_names=None,
        num_scenarios_per_type=None,
        limit_total_scenarios=None,
        timestamp_threshold_s=None,
        ego_displacement_minimum_m=None,
        expand_scenarios=True,
        remove_invalid_goals=False,
        shuffle=False
    )

    # 3. Try to get the scenarios
    worker = SingleMachineParallelExecutor(use_process_pool=True)
    scenarios = builder.get_scenarios(scenario_filter, worker)

    # 4. Print the result
    print("\n------ RESULT ------")
    print(f"SUCCESS: Found {len(scenarios)} scenarios!")
    print("--------------------")

except Exception as e:
    print("\n------ ERROR ------")
    import traceback
    traceback.print_exc()
    print("-------------------")

--- Minimal Test Script (v2) ---
Attempting to load data from: /home/bydguikong/nuplan/dataset/nuplan-v1.1/splits/mini
Attempting to load maps from: /home/bydguikong/nuplan/dataset/maps

------ RESULT ------
SUCCESS: Found 443218 scenarios!
--------------------


In [3]:
import os
import json

# --- 您需要修改这里的路径 ---
# 这个路径应该和您运行主脚本时使用的 --data_path 一致
data_directory = '/home/bydguikong/nuplan/dataset/nuplan-v1.1/splits/mini'
# data_directory = '/home/bydguikong/yy_ws/Diffusion-Planner/detect_data' 


# 新的json文件名
output_json_file = 'nuplan_train.json'
# --------------------------

actual_log_names = []
print(f"Scanning directory: {data_directory}...")

for root, dirs, files in os.walk(data_directory):
    for file in files:
        if file.endswith('.db'):
            # nuPlan的log_name通常不包含.db后缀
            # actual_log_names.append(file)
            log_name = os.path.splitext(file)[0]
            actual_log_names.append(log_name)

print(f"Found {len(actual_log_names)} log files.")

with open(output_json_file, 'w') as f:
    json.dump(actual_log_names, f, indent=4)

print(f"Successfully saved log names to {output_json_file}")

Scanning directory: /home/bydguikong/nuplan/dataset/nuplan-v1.1/splits/mini...
Found 64 log files.
Successfully saved log names to nuplan_train.json
